In [539]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re

%matplotlib inline

In [540]:
player_stats = pd.read_csv('baseballdatabank-master/core/Master.csv')
salaries = pd.read_csv('baseballdatabank-master/core/Salaries.csv')
fielding = pd.read_csv('baseballdatabank-master/core/Fielding.csv')
batting = pd.read_csv("baseballdatabank-master/core/Batting.csv")
pitching = pd.read_csv("baseballdatabank-master/core/Pitching.csv")
teams = pd.read_csv("baseballdatabank-master/core/Teams.csv")

In [541]:
player_salaries = pd.merge(player_stats, salaries)
master_stats = player_salaries[['playerID', 'nameGiven', 'deathYear','birthYear', 'salary', 'yearID', 'teamID']]

Remove duplicate playerID and take their most recent salary (not highest salary)

In [542]:
master_stats_no_dup = master_stats.drop_duplicates(['playerID'], take_last=True)

/usr/local/lib/python3.5/site-packages/ipykernel/__main__.py:1: FutureWarning: the take_last=True keyword is deprecated, use keep='last' instead
  if __name__ == '__main__':


Active Players (Current/Retired)

In [543]:
active_players = master_stats_no_dup[master_stats_no_dup.deathYear.isnull()]
# active_players

Calculations for OBP

In [544]:
calc_for_batting = batting[['playerID', 'H', 'BB', 'HBP', 'SF', 'AB', '2B', '3B', 'HR']].copy()
batting_stat = batting[['playerID', 'G']]
numerator_OBP = calc_for_batting[['H', 'BB', 'HBP']]
denominator_OBP = calc_for_batting[['AB', 'BB', 'HBP', 'SF']]
# calc_for_OBP.head(10)

In [545]:
numerator_OBP = numerator_OBP.sum(axis=1)
denominator_OBP = denominator_OBP.sum(axis=1)

In [546]:
calc_for_batting['numerator_OBP'] = numerator_OBP
calc_for_batting['denominator_OBP'] = denominator_OBP

In [547]:
OBP = calc_for_batting['numerator_OBP'] / calc_for_batting['denominator_OBP']

In [548]:
batting_stat['OBP'] = OBP

/usr/local/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


Runs Created - To create a measurement between hitting and pitching. This allows you to determine who is a better hitter depending on the number of people on bases

Total bases: one for each single, two for each double, three for each triple, and four for each home run 
[H + 2B + (2 * 3B) + (3 * HR)]

In [549]:
TB = calc_for_batting['H'] + calc_for_batting['2B'] + (2* calc_for_batting['3B']) + (3* calc_for_batting['HR'])

In [550]:
batting_stat['TB'] = TB

/usr/local/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [573]:
clean_batting = batting_stat[batting_stat.OBP.isnull()==False]

In [572]:
batting_stats = pd.merge(master_stats, clean_batting)

In [568]:
batting_stats[batting_stats.deathYear.isnull()]
batting_stats = batting_stats.drop_duplicates(['playerID'], take_last=True)
batting_stats = batting_stats[(batting_stats['OBP'] != 0) & (batting_stats['OBP'] > 0.4)]
batting_stats = batting_stats[(batting_stats['TB'] != 0) & (batting_stats['salary'] < 1000000)]
batting_stats = batting_stats[(batting_stats['yearID'] > 2010) & (batting_stats['yearID'] < 2015)]


/usr/local/lib/python3.5/site-packages/ipykernel/__main__.py:2: FutureWarning: the take_last=True keyword is deprecated, use keep='last' instead
  from ipykernel import kernelapp as app


In [571]:
batting_stats.sort_values(['TB', 'OBP'], ascending=[False, False])

,playerID,nameGiven,deathYear,birthYear,salary,yearID,teamID,G,OBP,TB
127946,lutzza01,Zachary Craig,NaN,1986.0,490040,2013,NYN,15,0.461538,8.0
140688,mesame01,Melquisedec,NaN,1987.0,490200,2013,NYA,5,0.428571,7.0
128132,lyonbr01,Brandon James,NaN,1979.0,750000,2013,NYN,15,1.000000,2.0
29391,burriem01,Emmanuel Allen,NaN,1985.0,625000,2012,SFN,5,0.800000,2.0
6410,augenbr01,Bryan Christopher,NaN,1986.0,414000,2011,SLN,5,1.000000,1.0
98234,horstje01,Jeremy Michael,NaN,1985.0,497000,2013,PHI,12,1.000000,1.0
139736,meekev01,Evan David,NaN,1983.0,875000,2012,PIT,70,1.000000,1.0
181586,rodrian01,Aneury,NaN,1987.0,414000,2011,HOU,1,1.000000,1.0
190535,schlibr01,Brian Patrick,NaN,1985.0,414000,2011,PHI,61,1.000000,1.0
5486,arredjo01,Jose Juan,NaN,1984.0,800000,2012,CIN,53,0.500000,1.0
